# **Instalação das bibliotecas**

In [ ]:
! pip install web3

In [ ]:
! pip install py-solc-x

# **Importação das bibliotecas e conexão com o Provedor**

In [ ]:
from web3 import Web3

Provedor Remoto (HTTP) disponibilizado pelo Alchemy: Sepolia testnet. Se retornar True, tudo ok. Caso contrário, verifique se estão corretos o tipo de conexão do provedor (IPC, HTTP, WebSocket) e a url.

In [ ]:
url = f"https://eth-sepolia.g.alchemy.com/v2/***REMOVED***"
w3 = Web3(Web3.HTTPProvider(url))
w3.is_connected()

# **Testando**

Teste a conexão com o provedor:

In [ ]:
w3.is_connected()

Extraia o último bloco aprovado na cadeia da Sepolia e verifique a corretude através do ***Etherscan***.

---

Observe que blocos são minerados muito rápido 😸

In [ ]:
ultimo_bloco_dict = w3.eth.get_block('latest')
ultimo_bloco_dict['number']

Agora, vamos instanciar nossa account para facilitar nosso deploy. Obtenha suas chaves através do ***MetaMask***.

In [ ]:
from web3 import Account
from google.colab import userdata
public_address = '***REMOVED***'
# Cria um objeto account a partir da chave privada. Isso é necessário para o deploy
account = Account.from_key('***REMOVED***')

# Verificando
assert account.address == public_address, "O endereço da conta gerada não coincide com o endereço público definido!"
print("A conta foi criada com sucesso e o endereço coincide!")

Cheque seu saldo. Não se assuste com o resultado😆!

In [ ]:
w3.eth.get_balance(account.address)

Se você ainda estiver pobre, carregue sua conta com https://cloud.google.com/application/web3/faucet/ethereum/sepolia e depois rode novamente a linha acima.

# **Compilação do contrato**
Para esta parte, é essencial que a biblioteca py-solc-x esteja instalada.

In [ ]:
from solcx import install_solc, get_installable_solc_versions, set_solc_version, get_solc_version, compile_source, compile_files
get_installable_solc_versions()

Instale o compilador:

In [ ]:
install_solc("0.8.20") # Insira a versão específica desejada, ou "latest" para instalar a mais recente.
set_solc_version("0.8.20")
get_solc_version()

Compile o contrato:

In [ ]:
codigo_fonte = '''
  pragma solidity 0.8.20;

  contract VendingMachine {

      // Declare state variables of the contract
      address public owner;
      mapping (address => uint) public cupcakeBalances;

      // When 'VendingMachine' contract is deployed:
      // 1. set the deploying address as the owner of the contract
      // 2. set the deployed smart contract's cupcake balance to 100
      constructor() {
          owner = msg.sender;
          cupcakeBalances[address(this)] = 100;
      }

      // Allow the owner to increase the smart contract's cupcake balance
      function refill(uint amount) public {
          require(msg.sender == owner, "Only the owner can refill.");
          cupcakeBalances[address(this)] += amount;
      }

      // Allow anyone to purchase cupcakes
      function purchase(uint amount) public payable {
          require(msg.value >= amount * 0.001 ether, "You must pay at least 0.001 ETH per cupcake");
          require(cupcakeBalances[address(this)] >= amount, "Not enough cupcakes in stock to complete this purchase");
          cupcakeBalances[address(this)] -= amount;
          cupcakeBalances[msg.sender] += amount;
      }
  }
'''
result = compile_source(codigo_fonte, output_values=['abi', 'bin'],  solc_version="0.8.20")
contract_id, contract_data = result.popitem()

abi = contract_data["abi"]
bytecode = contract_data["bin"]

ou

In [ ]:
with open('/content/contrato-maquina.txt', 'r') as file:
    codigo_fonte = file.read()
compile_source(codigo_fonte, output_values=['abi', 'bin'],  solc_version="0.8.20")

ou ainda

In [ ]:
compile_files('/content/contrato-maquina.sol', output_values=['abi', 'bin'],  solc_version="0.8.20")

# **Deploy na rede**

Instancie o contrato, utilizando a interface e o bytecode gerado pela compilação:

In [ ]:
contrato = w3.eth.contract(abi=abi, bytecode=bytecode)

Construa a transação de deploy:

In [ ]:
unsent_deploy_tx = contrato.constructor().build_transaction({
    'from': account.address,
    'gas': 1000000,
    'gasPrice': w3.eth.gas_price,
    'nonce': 8
})
print(unsent_deploy_tx)

Assine a transação utilizando sua chave privada:

In [ ]:
signed_deploy_tx = w3.eth.account.sign_transaction(unsent_deploy_tx, account.key)
print(signed_deploy_tx)

Envie a transação e confira o endereço do seu contrato. Busque esse endereço no ***Etherscan!***

In [ ]:
tx_hash = w3.eth.send_raw_transaction(signed_deploy_tx.raw_transaction)
endereco_contrato = w3.eth.wait_for_transaction_receipt(tx_hash)['contractAddress']
print(endereco_contrato)

# **Interação com o contrato**

In [ ]:
maquina = w3.eth.contract(address=endereco_contrato, abi=abi)
maquina.all_functions()

In [ ]:
unsent_tx = maquina.functions.purchase(1).build_transaction({
    'from': '***REMOVED***',
    'nonce': w3.eth.get_transaction_count('***REMOVED***'),
    'gas': 100000,
    'gasPrice': w3.eth.gas_price,
    'value': w3.to_wei(0.001, 'ether')
    })
signed_tx = w3.eth.account.sign_transaction(unsent_tx, '***REMOVED***')
tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
w3.eth.wait_for_transaction_receipt(tx_hash)

In [ ]:
maquina.functions.owner().call()

In [ ]:
maquina.functions.refill(10).build_transaction({
    'from': account.address,
    'nonce': w3.eth.get_transaction_count('***REMOVED***'),
    'gas': 100000,
    'gasPrice': w3.eth.gas_price,
    })
signed_tx = w3.eth.account.sign_transaction(unsent_tx, account.key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
w3.eth.wait_for_transaction_receipt(tx_hash)